In [1]:
using JuMP
using Gurobi
using CSV
using DataFrames

const GRB_ENV = Gurobi.Env(output_flag=1);

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-21


In [ ]:
facilities = CSV.read("data/Demand.csv",DataFrame);
food_prices = CSV.read("data/Food_Prices.csv",DataFrame);
warehouses = CSV.read("data/Warehouses.csv",DataFrame);
transportation = CSV.read("data/Transportation_costs.csv",DataFrame) |> Matrix;
nutritional_value = CSV.read("data/Nutritional_Value.csv",DataFrame) |> Matrix;
nutrients = CSV.read("data/Nutrients.csv",DataFrame);

# Extract data from DataFrames into arrays for easier indexing
D = facilities[!, :Population] |> Array; # demand for each facility location j (vector, 27)
D = D[1:end, 1:end] * 0.01; # fraction of population for now that makes solution feasible
t = transportation[1:end, 1:end .!= 1]; # removing extra column from transportation costs (matrix, 10 x 27)
π = food_prices[!, :PriceFINAL] |> Array; # food prices per item k (vector, 14)
V = parse.(Float64,string.(nutritional_value[1:end, 1:end .!= 1])) .* 10 |> Array; # nutritional value for the different types of foods (matrix, 14 x 5)
N = nutrients[!, :Requirements] |> Array; # daily requirements for select nutrients (vector, 5)
C = 1000000000000 # max capacity constraint
S = 10000 # cost to build a facility

# variable index sizes
m = size(t)[1]; # number of warehouses, 10
n = size(t)[2]; # number of facilities (demand), 27
p = size(π)[1]; # number of food items, 14
q = size(N)[1]; # number of nutrient requirements, q

## Model formulation without uncertainty (nominal value of transportation costs)

In [4]:
# parameter to change
M = 15;

# Model definition
model = Model(() -> Gurobi.Optimizer(GRB_ENV))
set_optimizer_attribute(model,"Method",0)

# Decision variables
@variable(model, y[1:n], Bin)  # y[j]: binary decision variable for facility j
@variable(model, x[1:m, 1:n, 1:p] >= 0)  # x[i, j, k]: quantity transported from i to j of food k

# Objective function: minimize total cost
@objective(model, Min, 
    sum(π[k] * x[i, j, k] for i in 1:m, j in 1:n, k in 1:p) +
    sum(t[i, j] * x[i, j, k] for i in 1:m, j in 1:n, k in 1:p) +
    S * sum(y[j] for j in 1:n))

# Constraints
@constraint(model, [j=1:n, k=1:p], sum(x[i, j, k] for i in 1:m) >= D[j]) # meet all demands for each population j
@constraint(model, [l=1:q], sum(V[k, l] for k in 1:p) >= N[l]) # meet daily nutrient requirements
@constraint(model, [k=1:p, j=1:n], sum(x[i, j, k] for i in 1:m) <= C * y[j]) # do not exceed facility capacity, IGNORE
@constraint(model, sum(y[j] for j in 1:n) >= M) # must build at least M facilities

# Solve the model
optimize!(model)

# Output the results
if termination_status(model) == MOI.OPTIMAL
    println("Optimal total cost: ", objective_value(model))
    selected_facilities = findall(value.(y) .> 0.5)  # Indices of nonzero y
    println("Selected facilities (indices): ", selected_facilities)
else
    println("No optimal solution found.")
end


Set parameter Method to value 0
Set parameter Method to value 0
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads
Optimize a model with 762 rows, 3807 columns and 7965 nonzeros
Model fingerprint: 0x28c43f0d
Variable types: 3780 continuous, 27 integer (27 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+12]
  Objective range  [1e+00, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+04]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 2534814.1195
Presolve removed 762 rows and 3807 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 11 available processors)

Solution count 2: 443035 2.53481e+06 

Optimal solution found (tolerance 1.00e-04)
         (possib

## Results for nominal value model

M = 1:
Optimal total cost: 383034.9090967625
Selected facilities (indices): [3, 5, 10, 11, 12, 13, 14, 15, 16]

M = 10:
Optimal total cost: 393034.9090967625
Selected facilities (indices): [3, 5, 10, 11, 12, 13, 14, 15, 16, 27]

M = 15:
Optimal total cost: 443034.9090967625
Selected facilities (indices): [3, 5, 10, 11, 12, 13, 14, 15, 16, 22, 23, 24, 25, 26, 27]

M = 20:
Optimal total cost: 493034.9090967625
Selected facilities (indices): [3, 5, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]

M = 24:
Optimal total cost: 533034.9090967625
Selected facilities (indices): [3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]

M = 27:
Optimal total cost: 563034.9090967625
Selected facilities (indices): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
